# Preprocesamiento de Datos
---
Este notebook documenta el proceso de preprocesamiento de datos para un conjunto de datos relacionado con la predicción de bancarrotas de empresas. A lo largo de este flujo de trabajo, se realizan diversas etapas, como la descarga del dataset, limpieza de datos, balanceo de clases, selección de características y almacenamiento de los datos procesados. El objetivo principal es preparar los datos para su uso en modelos de aprendizaje automático, asegurando que estén limpios, balanceados y contengan las características más relevantes para el análisis

### Importación de librerías

In [1]:
import kagglehub
import pandas as pd
import json

c:\Users\david\Documents\CLOUD-COMPUTING.Actividad-Semana-2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Descarga de dataset desde KaggleHub

In [2]:
print("Descargando dataset...")
path = kagglehub.dataset_download("fedesoriano/company-bankruptcy-prediction")

Descargando dataset...


100%|██████████| 4.63M/4.63M [00:00<00:00, 10.6MB/s]

Extracting files...


### Lectura del archivo de CSV

In [3]:
data_path = path + "/data.csv"
df = pd.read_csv(data_path)

print(f"Dataset cargado exitosamente. Dimensiones: {df.shape}")
df.head()

Dataset cargado exitosamente. Dimensiones: (6819, 96)


,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


In [4]:
# Información general del dataset
print("Resumen de columnas y tipos de datos:")
df.info()

Resumen de columnas y tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6819 entries, 0 to 6818
Data columns (total 96 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   Bankrupt?                                                 6819 non-null   int64  
 1    ROA(C) before interest and depreciation before interest  6819 non-null   float64
 2    ROA(A) before interest and % after tax                   6819 non-null   float64
 3    ROA(B) before interest and depreciation after tax        6819 non-null   float64
 4    Operating Gross Margin                                   6819 non-null   float64
 5    Realized Sales Gross Margin                              6819 non-null   float64
 6    Operating Profit Rate                                    6819 non-null   float64
 7    Pre-tax net Interest Rate                                6819 non-

### Limpieza de Datos

In [5]:
# Eliminamos filas con valores nulos
df_clean = df.dropna()

print(f"\nDimensiones después de eliminar filas con nulos: {df_clean.shape}")


Dimensiones después de eliminar filas con nulos: (6819, 96)


### Balanceo con oversampling

In [6]:
# Verificamos balance de la variable objetivo
print("Distribución de la variable objetivo (Bankrupt?):")
print(df_clean["Bankrupt?"].value_counts(normalize=True))

Distribución de la variable objetivo (Bankrupt?):
Bankrupt?
0    0.967737
1    0.032263
Name: proportion, dtype: float64


In [7]:
from sklearn.utils import resample

# Separar por clase
bankrupt = df_clean[df_clean["Bankrupt?"] == 1]
not_bankrupt = df_clean[df_clean["Bankrupt?"] == 0]

# Aumentamos la clase minoritaria al tamaño de la mayoritaria 
bankrupt_oversampled = resample(bankrupt,
                                replace=True,      # muestra con reemplazo
                                n_samples=len(not_bankrupt),  # para igualar el tamaño
                                random_state=42)

# Combinamos los datos balanceados
df_balanced = pd.concat([not_bankrupt, bankrupt_oversampled]).sample(frac=1, random_state=42)

# Verificamos la nueva distribución de la variable objetivo
print(df_balanced["Bankrupt?"].value_counts(normalize=True))


Bankrupt?
0    0.5
1    0.5
Name: proportion, dtype: float64


### Eliminar columnas NO Relevantes

In [8]:
from sklearn.feature_selection import SelectKBest, f_classif

X = df_balanced.drop("Bankrupt?", axis=1)
y = df_balanced["Bankrupt?"]

In [9]:
from sklearn.feature_selection import VarianceThreshold

# Eliminar columnas constantes o casi constantes
print(f"Dimensiones antes de eliminar constantes: {X.shape}")
selector_var = VarianceThreshold(threshold=0.0)
X_var = selector_var.fit_transform(X)

# Obtener nombres de las columnas no constantes
non_constant_columns = X.columns[selector_var.get_support()]
X = pd.DataFrame(X_var, columns=non_constant_columns)
print(f"Dimensiones después de eliminar constantes: {X.shape}")

Dimensiones antes de eliminar constantes: (13198, 95)
Dimensiones después de eliminar constantes: (13198, 94)


In [10]:
# Seleccionar las 7 mejores características utilizando librería de selección de características
selector = SelectKBest(score_func=f_classif, k=7)
X_new = selector.fit_transform(X, y)

# Obtener nombres de columnas seleccionadas
selected_columns = X.columns[selector.get_support()]
print("Variables seleccionadas:", selected_columns)

selected_columns_json = selected_columns.tolist()
with open("selected_columns.json", "w") as f:
    json.dump(selected_columns_json, f)

# Crear nuevo DataFrame con las variables seleccionadas y la variable objetivo
df_selected = pd.concat([pd.DataFrame(X_new, columns=selected_columns), y.reset_index(drop=True)], axis=1)

Variables seleccionadas: Index([' ROA(C) before interest and depreciation before interest',
       ' ROA(B) before interest and depreciation after tax',
       ' Persistent EPS in the Last Four Seasons',
       ' Per Share Net profit before tax (Yuan ¥)', ' Debt ratio %',
       ' Net worth/Assets', ' Net profit before tax/Paid-in capital'],
      dtype='object')


### Guardamos Datos en Archivo

In [11]:
df_selected.head()

,ROA(C) before interest and depreciation before interest,ROA(B) before interest and depreciation after tax,Persistent EPS in the Last Four Seasons,Per Share Net profit before tax (Yuan ¥),Debt ratio %,Net worth/Assets,Net profit before tax/Paid-in capital,Bankrupt?
0,0.521620,0.567803,0.234944,0.187542,0.121928,0.878072,0.186555,0
1,0.437089,0.470207,0.185024,0.148753,0.207576,0.792424,0.147772,1
2,0.549798,0.601156,0.256500,0.208255,0.140067,0.859933,0.206960,0
3,0.446644,0.494887,0.201853,0.161106,0.153793,0.846207,0.160094,1
4,0.506021,0.564645,0.231918,0.183551,0.195090,0.804910,0.181977,1


In [12]:
df_selected.to_csv("data_selected.csv", index=False)
print("Dataset con variables seleccionadas guardado como 'data_selected.csv'")

Dataset con variables seleccionadas guardado como 'data_selected.csv'
